# 16.1 生成器如何进化成协程

PEP 380 对生成器函数的语法做了两处改动，以便更好地作为协程使用:
* 现在，生成器可以返回一个值: 以前，如果在生成器中给 return 语句提供值，会抛出 SyntaxError 异常。
* 新引入 yield from 语法，使用它可以把复杂的生成器冲构成小型的嵌套生成器，省去了之前把生成器的工作委托给子生成器所需的大量样板代码

# 16.2 用作协程的生成器的基本行为

In [6]:
# 协程的简单使用演示
def simple_coroutine():
    print('-> coroutine started')
    x = yield
    print('-> coroutine recived:', x)

my_coro = simple_coroutine()
my_coro

<generator object simple_coroutine at 0x000001C11D9DF120>

In [7]:
next(my_coro)

-> coroutine started


In [8]:
my_coro.send(42)

-> coroutine recived: 42


StopIteration: 

协程可以身处四个状态中的一个。当前状态可以使用 inspect.getgeneratorstate(...) 函数确定:
* 'GEN_CREATED': 等待开始执行
* 'GEN_RUNNING': 解释器正在执行
* 'GEN_SUSPENDED': 在 yield 表达式处暂停
* 'GEN_CLOSED': 执行结束

因为 send 方法的参数会成为暂停的 yield 表达式的值，所以，仅当协程处于暂停状态时才可以调用 send 方法。如果协程还没激活，情况就不同了。所以在创建协程时一定要使用 next(gen) 激活协程，也可以调用 gen.send(None) 效果一样。

最先调用 next(gen) 函数的这一步通常称为 预激(prime) 协程

In [9]:
# 产出两个值的协程
def simple_coco2(a):
    print('-> Started: a=', a)
    b = yield a
    print('-> Received: b=', b)
    c = yield a + b
    print('-> Received: c=', c)

my_coro = simple_coco2(14)
from inspect import getgeneratorstate
getgeneratorstate(my_coro)

'GEN_CREATED'

In [10]:
next(my_coro)

-> Started: a= 14


14

In [11]:
getgeneratorstate(my_coro)

'GEN_SUSPENDED'

In [12]:
my_coro.send(28)

-> Received: b= 28


42

In [13]:
my_coro.send(99)

-> Received: c= 99


StopIteration: 

# 16.3 示例: 使用协程计算移动平均值

In [14]:
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

# 16.4 预激协程的装饰器

In [15]:
# 预激协程的装饰器
from functools import wraps

def corountine(func):

    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return primer

# 16.5 终止协程和异常处理

In [16]:
# 未处理的异常会导致协程中止
@corountine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

coro_avg= averager()
coro_avg.send(40)

40.0

In [17]:
coro_avg.send(50)

45.0

In [18]:
coro_avg.send('spam')

TypeError: unsupported operand type(s) for +=: 'float' and 'str'

In [19]:
coro_avg.send(40)

StopIteration: 

从 Python 2.5 开始，客户代码可以在生成器对象上调用两个方法，显式地把异常发给协程，这两个方法是 throw 和 close

generator.throw(exc_type[, exc_value[, traceback])  
致使生成器在暂停的 yield 表达式处抛出指定的异常。如果生成器处理了抛出的异常，代码会向前执行到下一个 yield 表达式，而产出的值会变成 generator.throw 方法得到的返回值。如果生成器没有处理抛出的异常，异常会向上冒泡，传到调用方的上下文中。

generator.close()  
致使生成器在暂停的 yield 表达式抛出 GeneratorExit 异常。如果生成器没有处理这个异常或者抛出了 StopIteration 异常(通常是运行到结尾)，调用方不会报错。如果收到 GeneratorExit 异常，生成器一定不能产出值，否则解释器会抛出 RuntimeError 异常。

In [26]:
# 在协程中处理异常的测试代码
class DemoException(Exception):
    """演示所用的异常类型"""

def demo_exc_handling():
    print('-> corotine started')
    while True:
        try:
            x = yield
        except DemoException:
            print('OKOK')
        else:
            print('-> coroutine received: {!r}'.format(x))
    raise RuntimeError('This line should never run.')

In [21]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> corotine started


In [23]:
exc_coro.send(11)
exc_coro.send(22)

-> coroutine received: 11
-> coroutine received: 22


In [25]:
exc_coro.close()
from inspect import getgeneratorstate
getgeneratorstate(exc_coro)

'GEN_CLOSED'

In [27]:
# 把 DemoExcption 异常传入 demo_exc_handling 不会导致协程中止
exc_coro = demo_exc_handling()
next(exc_coro)
exc_coro.throw(DemoException)

-> corotine started
OKOK


In [28]:
# 如果传入无法处理的异常 协程会中止
exc_coro = demo_exc_handling()
next(exc_coro)
exc_coro.throw(TypeError)

-> corotine started


TypeError: 

如果不管协程如何结束都想做清理工作，要把协程定义体相关内容放入 try/finally 块中

In [ ]:
class DemoException(Exception):
    """演示所用的异常类型"""

def demo_finally():
    print('-> corotine started')
    try:
        while True:
            try:
                x = yield
            except DemoException:
                print('OKOK')
            else:
                print('-> coroutine received: {!r}'.format(x))
    finally:
        print('ending')

# 16.6 让协程返回值

In [30]:
# 可以返回值的协程
from collections import namedtuple

Result = namedtuple('Result', 'count average')  # 项数和平均值

def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)
    

In [31]:
# 使用新版 averager
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)
coro_avg.send(None)

StopIteration: Result(count=3, average=15.5)

发送 None 会终止循环，导致协程结束，返回结果

注意：return 表达式的值会偷偷传给调用方，赋值给 StopIteration 异常的一个属性。这样做有点不合常理，但是能保留生成器对象的常规行为 —— 耗尽时抛出 StopIteration 异常

In [32]:
# 获取 averager 返回的值
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)
try:
    coro_avg.send(None)
except StopIteration as exc:
    result = exc.value
result

Result(count=3, average=15.5)

虽然饶了一个圈子，但这是 PEP 380 定义的方式，而这也表明，yield from 结构会在内部自动捕获 StopIteration 异常并将 value 属性的值变成外部 yield from 表达式的值

# 16.7 使用 yield from

在生成器 gen 中使用  yield from subgen() 时，subgen() 获得控制权，把产出的值传给 gen 的调用方，即调用方可以直接控制 subgen。与此同时，gen 会阻塞，等待 subgen 停止

In [33]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1, 3):
        yield i
list(gen())

['A', 'B', 1, 2]

In [35]:
# 可以改写为
def gen():
    yield from 'AB'
    yield from range(1, 3)

list(gen())

['A', 'B', 1, 2]

yield from x 表达式对 x 对象所做的第一件事是，调用 iter(x), 从中获取迭代器。因此，x 可以是任何可迭代对象

可是，如果 yield from 结构唯一的作用是替代产出值的嵌套 for 循环，这个结构很有可能不会添加到 Python 语言中。yield from 结构的本质作用无法通过简单的可迭代对象说明，而要散发四维，使用嵌套的生成器。

yield from 的主要功能是打开双向通道，把最外层的调用方与最内层的子生成器连接起来，这样而这可以直接发送和产出值，还可以直接传入异常，而不用在位于中间的协程中添加大量处理异常的样板代码。有了这个结构，协程可以通过以前不可能的方式委托职责

下面是一些专门术语: 
* 委派生成器: 包含 yield from <iterable> 的生成器函数 
* 子生成器: 从 <iterable> 部分获取的生成器
* 调用方: 调用委派生成器的客户端代码

In [39]:
# 从一个字典中读取虚构的男女学生的体重和身高，然后生成一个报告

from collections import namedtuple

Result = namedtuple('Result', 'count average')  # 项数和平均值

# 子生成器
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield # main 函数中的客户代码发送的各个值绑定到这里的 term 变量上
        if term is None:    # 终止条件
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average) # 返回的 Result 会称为 grouper 函数中 yield from 表达式的值

# 委派生成器
def grouper(results, key):
    while True: # 这个循环每次迭代时会新建一个 averager 实例，每个实例都是作为协程使用的生成器对象
        # 写死循环的原因是防止在赋值后生成器结束，抛出 StopIteration 异常
        results[key] = yield from averager()

# 客户端代码,即调用方
def main(data):
    results = {}
    for key,values in data.items():
        group = grouper(results, key)   
        # group 是调用 grouper 函数得到的生成器对象，传给 grouper 函数的第一个参数是 results，用于收集结果
        next(group) # 预激 group
        for value in values:    
            # 把各个 value 传给 grouper，传入的值最终到达 averager 函数中 term = yield 那一行，grouper 永远不知道传入的值是什么
            group.send(value)
        group.send(None)    # 当前 averager 实例中止，grouper继续运行，再创建一个 averager 实例，处理下一组值
    # print(results)
    report(results)

# 输出报告
def report(results):
    for key,result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(
            result.count, group, result.average, unit
        ))

data={ 'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}

if __name__ == '__main__':
    main(data)


 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m


* 外层 for 循环每次迭代会新建一个 grouper 实例，赋值给 group 变量，group 是委派生成器
* 调用 next(group)，预激委派生成器 grouper，此时进入 while True 循环，调用子生成器 averager 后，在 yield from 表达式处暂停
* 内层 for 循环调用 group.send(value)，直接把值传给 子生成器 avrager。同时，当前的 grouper 实例在 yield from 处暂停
* 内层循环结束后，group 实例依旧在 yield from 表达式处暂停，因此 grouper 函数定义体中为 results[key] 赋值的语句还未执行
* 如果外层 for 循环的末尾没有 group.send(None), 那么 averager 子生成器永远不会中止，委派生成器 group 永远不会再次激活，因此永远不会为 results[key] 赋值
* 外层 for 循环重新迭代时会新建一个 grouper 实例，绑定在 group 变量上，前一个 grouper 实例被垃圾回收程序回收

# 16.8 yield from 的意义

yield from 的行为:
* 子生成器产出的值都会直接传给委派生成器的调用方(即客户端代码)
* 使用 send() 方法发给委派生成器的值都直接传给子生成器。如果发送的值是 None，那么会调用子生成器的 \_\_next__() 方法。如果发送的值不是 None，那么会调用子生成器的 send() 方法。如果调用的方法抛出 StopIteration 异常，那么委派生成器恢复运行，任何其他异常都会向上冒泡，传给委派生成器
* 生成器退出时，子生成器中的 return expr 表达式 会触发 StopIteration(expr) 异常抛出。
* yield from 表达式的值是子生成器终止时传给 StopIteration 异常的第一个参数

yield from 结构的另外两个特性与异常和终止有关
* 传入委派生成器的异常，除了 GeneratorExit 之外都传给子生成器的 throw() 方法。如果调用 therow() 方法时抛出 StopIteration 异常，委派生成器恢复运行。StopIteration 之外的异常会向上冒泡，传给委派生成器。
* 如果把 GeneratorExit 异常传入委派生成器，或者在委派生成器上调用 close() 方法，那么在子生成器上调用 close() 方法，如果它有实现的话。如果调用 close() 方法导致异常抛出，那么异常会向上冒泡，传给委派生成器；否则，委派生成器抛出 GeneratorExit 异常

假设 yield from 出现在委派生成器中。客户端代码驱动着委派生成器，而委派生成器驱动子生成器。那么，为了简化逻辑，我们假设客户端没有在委派生成器上调用 .throw() 或 .close() 方法。此外，我们还假设子生成器不会抛出异常，而是一直运行到中止，抛出 StopIteration 异常。

```python
# 简化的伪代码，等效于委派生成器中 RESULT = yield from EXPR 语句
_i = iter(EXPR) # EXPR 可以是任意可迭代的对象，因为获取迭代器 _i 使用的是 iter() 函数

try:
    _y = next(_i) # 预激子生成器，结果保存在 _y 中，作为产出的第一个指
except StopIteration as _e:
    _r = _e.value # 如果抛出 StopIteration 异常，获取其 value 属性，赋值给 _r ———— 这是最简单情况下的返回值
else:
    while 1: # 运行这个循环时，委派生成器会阻塞，只作为调用方和子生成器之间的通道
        _s = yield _y # 产出子生成器当前产出的元素，等待调用方发送 _s 中保存的值
    try:
        _y = _i.send(_s) # 尝试让子生成器向前执行，转发调用方发送的 _s
    except StopIteration as _e: # 如果子生成器抛出 StopIteration 异常，获取 value 属性的值，赋值给 _r，然后退出循环，让委派生成器恢复运行
        _r = _e.value
        break

RESULT = _r # 返回的结果是 _r，即整个 yield from 表达式的值
```
这段代码的变量:
* _i (迭代器) : 子生成器
* _y (产出的值) : 子生成器产出的值(即子生成器运行结束后 yield from 表达式的值)
* _r (结果) : 最终的结果
* _s (发送的值) : 调用方发给委派生成器的值，这个值会转发给子生成器
* _e (异常) : 异常对象、

除了没有处理 throw 和 colse 方法之外，这段简化的伪代码还在子生成器上调用 .send 方法，以此达到客户调用 next 函数或 send 方法的目的。

但是，现实情况要复杂一些，因为要处理客户对 throw 和 close 方法的调用，而这两个方法执行的操作必须传入子生成器。此外，子生成器可能只是纯脆的迭代器，不支持 throw 和 close 方法，因此 yield from 结果的逻辑必须处理这种情况。如果子生成器实现了这两个方法，而在子生成器内部，这两个方法都会触发异常抛出，这种情况也必须由 yield from 机制处理。调用方可能会无缘无故地让子生成器自己抛出异常，实现 yield from 结果也必须处理这种情况。最后，为了优化，如果调用方调用 next 函数或 send(None) 方法，都要转交职责，在子生成器上调用 next 函数，仅当调用方法发送的值不是 None 时，才使用子生成器的 send 方法

下面列出 PEP 380 中扩充 yield from 表达式的完整伪代码，它等效于委派生成器中的 RESULT = yield from EXPR 语句

```python
_i = iter(EXPR) # EXPR 可以是任意可迭代对象，因为获取迭代器 _i 使用的是 iter() 函数
try:
    _y = next(_i) # 预激子生成器；结果保存在 _y 中，作为产出的第一个值
except StopIteration as _e:
    _r = _e.value # 如果抛出 StopIteration 异常，获取异常对象的 value 属性，赋值给 _r，即返回值
else:
    while 1: # 运行这个循环时，委派生成器会阻塞，只作为调用方和子生成器之间的通信
        try:
            _s = yield _y # 产出子生成器当前产出的元素；等待调用方发送 _s 中保存的值
        except GeneratorExit as _e: # 这一部分用于关闭委派生成器和子生成器。因为子生成器可以是任何可迭代的对象，可能没有 close 方法
            try:
                _m = _i.close
            except AttributeError: 
                pass
            else: 
                _m()
            raise _e
        except BaseException as _e: 
            # 这一部分处理调用方法通过 .throw(...) 传入的异常。
            # 同样，子生成器可以是迭代器，从而没有 throw 方法可调用，这种情况会导致委派生成器抛出异常
            _x = sys.exc_info() 
            try:
                _m = _i.throw 
            except AttributeError:
                raise _e 
            else: 
                # 如果子生成器有 throw 方法，则传入异常。
                # 子生成器可能会处理传入的异常(然后继续循环)，可能抛出 StopIteration 异常(从中获取结果，赋值给 _r)
                # 还可能不处理，而是抛出相同或不协调的异常，向上冒泡，传给委托生成器
                try:
                    _y = _m(*_x)
                except StopIteration as _e: 
                    _r = _e.value
                    break 
        else: # 如果产出值时没有异常
            try: # 尝试让子生成器向前执行
                if _s is None: # 如果调用方法最后发送的是 None，则在子生成器上调用 next 函数，否则将 _s 传入子生成器
                    _y = next(_i) 
                else:
                    _y = _i.send(_s) 
            except StopIteration as _e: # 如果子生成器抛出 StopIteration 异常，获取 value 的属性，赋值给 _r，然后退出循环，让委派生成器继续执行
                _r = _e.value 
                break

RESULT = _r # 返回的结果是 _r，即整个 yield from 表达式的值
```

注意：用于预激的装饰器 与 yield from 结果不兼容


# 16.9 使用案例: 使用协程做离散事件仿真

## 16.9.1 离散事件仿真简介

离散事件仿真(Discrete Event Simulation, DES)是以中把系统建模成一系列事件的仿真类型，在离散事件仿真中，仿真“钟”向前推进的量是不固定的，而是直接推进到下一个事件模型的模拟时间。使用 DES 可以在不到一秒钟的时间内模拟一年的出租车运营过程。这与连续仿真不同，连续放真的仿真钟以固定的量不断向前推进。

## 16.9.2 出租车对运营仿真

仿真程序会创建几辆出租车，每辆车会拉几个乘客，然后回家，出租车首先驶离车库，四处徘徊，寻找乘客；拉到乘客后，形成开始；乘客下车后继续四处徘徊

* 出租车每隔 5 分钟从车库出发
* 所有排定的时间都在默认的仿真时间内(180 分钟)完成，最后一次事件发生在 time = 110 时。
* 仿真结束时可能还有未完成的事件。如果是这种情况，最后一条消息会是这样: 
\*** end of simulation time: 3 events pending ***
* 这不是完整代码，只列出了与协程相关的部分。真正重要的函数只有两个 taxi_process(一个协程)，以及执行仿真主循环的 Simulator.run 方法
* 这个协程用到了别处定义的两个对象: compute_delay函数，返回单位为分钟的间隔时间: Event 类，一个 namedtuple，定义方式如下:  
  Event = collections.namedtuple('Event', 'time proc action')
  * time: 事件发生时的仿真时间
  * proc: 出租车进程实例的编号
  * action: 描述活动的字符串

In [40]:
# taxi_process 协程，实现各辆出租车的活动
import collections

Event = collections.namedtuple('Event', 'time proc action')

def taxi_process(ident, trips, start_time=0):  
    """每次改变状态时创建时间，把控制权让给仿真器""" 
    # 每辆出租车调用一次 taxi_process 函数，创建一个生成器对象，表示各辆出租车的运营过程
    # ident 是出租车的编号；trips 是出租车回家之前的行程数量；start_time 是出租车离开车库的时间  
 
    # 产出的第一个 Event 是 leave garage。执行到这一行，协程会暂停，让仿真主循环着手处理下一个事件。
    # 需要激活这个进程时，主循环会发送当前的仿真时间，赋值给 time
    time = yield Event(start_time, ident, 'leave garage')

    # 每次行程都会执行这个代码块
    for i in range(trips):
        # 产出一个 Event 实例，表示拉到乘客了。协程在这里暂停
        time = yield Event(time, ident, 'pick up passenger') 
        # 产出一个 Event 实例，表示乘客下车了
        time = yield Event(time, ident, 'drop off passenger')

    # 指定的形参数量完成后，for 循环结束，最后产出 going home事件，此时，协程最后一次暂停。
    # 仿真主循环发送时间后，协程重新激活；不过这里没有把产出的值赋值给变量，因为用不到了
    yield Event(time, ident, 'going home') 
    # 出租车进程结束，即协程执行到最后，生成器对象抛出 StopIteration 异常

In [41]:
# 驱动 taxi_process 协程
taxi = taxi_process(ident=13, trips=2, start_time=0)
next(taxi)

Event(time=0, proc=13, action='leave garage')

In [42]:
# 现在可以发送当前时间。在控制台中，_ 变量绑定的是前一个结果；
# 这里在时间上加 7，意思是这俩出租车 7 分钟后找道第一个乘客
taxi.send(_.time + 7)

Event(time=7, proc=13, action='pick up passenger')

In [44]:
taxi.send(_.time + 23)

Event(time=30, proc=13, action='drop off passenger')

In [45]:
taxi.send(_.time + 5)

Event(time=35, proc=13, action='pick up passenger')

In [46]:
taxi.send(_.time + 48)

Event(time=83, proc=13, action='drop off passenger')

In [47]:
taxi.send(_.time + 1)

Event(time=84, proc=13, action='going home')

In [48]:
taxi.send(_.time + 10)

StopIteration: 

在这个仿真系统中，各个出租车由 Simulator.run 方法中的主循环驱动，仿真钟保存在 sim_time 变量中，每次产出事件时都会更新仿真钟。

为了实例化 Simulator 类，脚本的 main 函数构建了一个 taxis 字典

In [ ]:
DEPARTURE_INTERVAL = 5 # 发车间隔
taxis = {i: taxi_process(i, (i+1) * 2, i * DEPARTURE_INTERVAL) for i in range(num_taxis)}
sim = Simulator(taxis)

taxis 字典的值是三个参数不同的生成器对象。  
Simulator 的初始化方法如下，它的主要数据结构是:
* self.events : Priority 对象，保存 Event 实例。元素可以放进 PriorityQueue 对象中，然后按 item[0](即 Event 对象的 time 属性) 依次取出
* self.procs : 一个字典，拔出租租车的编号映射到仿真过程中激活的进程。这个属性会绑定前面所示的 taxis 字典副本

In [ ]:
# Simlator 类的初始化方法
import queue
class Simlator:

    def __init__(self, procs_map):
        self.events = queue.PriorityQueue()
        self.procs = dict(procs_map)

优先队列 PriorityQueue 是离散仿真系统的基础构件: 创建事件的顺序补丁，放入这种队列后，可以按照各个事件排定的顺序取出。

下面分析这个仿真系统的住算法 ———— Simulator.run 方法。在 main 函数中，实例化 Simulator 类之后立即就调用了这个方法
```python
    sim = Simulator(taxis)
    sim.run(end_time)
```
Simulator.run 方法实现的算法:
1. 迭代表示各辆出租车的进程
   * 在各辆出租车上调用 next() 函数，预激协程。这样会产出各辆出租车的第一个事件
   * 把各个事件放入 Simulator 类的 self.events 队列中
2. 满足 sim_time < end_time 条件时，运行仿真系统的主循环
   * 检查 self.events 属性是否为空：如果为空，跳出循环
   * 从 self.events 中获取当前事件 (current_event),即 PriorityQueue 对象中时间值最小的 Event 对象。
   * 显示获取的 Event 对象
   * 获取 current_event 的 time 属性，更新仿真时间
   * 把时间发给 current_event 的 proc 属性标识的协程，产出下一个事件 (next_event)
   * 把 next_event 添加到 self.events 队列中，排定 next_event

In [ ]:
class Simulator:

    def __init__(self, procs_map):
        self.events = queue.PriorityQueue()
        self.procs = dict(procs_map)

    def run(self, end_time):  # run 方法只需要仿真结束时间 end_time 这一个参数
        """排定并显示事件，直到时间结束"""

        # 排定各辆出租车的第一个事件
        for _, proc in sorted(self.procs.items()):  # 使用 sorted 获取 self.procs 中按键排序的元素，用不到键，所以赋值给 _
            first_event = next(proc)  # 调用 next 预激各个协程，向前执行到第一个 yield 表达式，做好接收数据的准备。产出一个 event 对象
            self.events.put(first_event)  # 把各个事件添加到 self.events 属性表示的 PriorityQueue 对象中。

        # 仿真系统的主循环
        sim_time = 0  # 把仿真钟归零
        while sim_time < end_time:  # 在结束之前持续运行
            if self.events.empty():  # 如果队列中无未完成的事件，则退出主循环
                print('*** end of events ***')
                break

            current_event = self.events.get()  # 获取优先队列中 time 属性最小的 Event 对象
            sim_time, proc_id, previous_action = current_event  # 拆包 Event 对象的数据，这一行代码会更新仿真钟，对应于事件发生的事件
            print('taxi:', proc_id, proc_id * '   ', current_event)  # 打印 Event 对象，指明是哪辆车，并根据车的编号缩进
            active_proc = self.procs[proc_id]  # 从 self.procs 字典钟获取当前活动的出租车的协程。

            # 调用 compute_duration(...) 函数，传入前一个动作，把结果加到 sim_time 上，计算下一次活动的时间
            next_time = sim_time + compute_duration(previous_action)
            try:
                # 把计算得到的时间发给出租车协程。协程会产出下一个事件(next_event)，或者抛出 StopIteration 异常
                next_event = active_proc.send(next_time)  
            except StopIteration:
                del self.procs[proc_id]  # 如果抛出 StopIteration 异常，代表收车，从字典中删除那个协程
            else:
                self.events.put(next_event)  # 否则将 next_event 放入队列中
        else:  # 如果循环由于仿真时间到了而退出，显示待完成的事件数量(有时可能碰巧是0)
            msg = '*** end of simulation time: {} events pending ***'
            print(msg.format(self.events.qsize()))

注意：上述代码中两处用到了 else 块，而且都不在 if 语句中:
1. 主 while 循环有一个 else 语句，报告仿真系统由于达到结束时间而结束，而不是由于没有事件处理而结束
2. 靠近主 while 循环底部的 try 语句把 next_time 发给当前的出租车进程，尝试获取下一个事件，如果成功，执行 else 块，把 next_event 放入队列中